# Python Course - Tutorial 10

### Exercise 1: Generator for a Simulated Price Path (Geometric Brownian Motion)

In empirical finance, asset prices are often modeled as stochastic processes. In this exercise, you will implement a generator that produces a simulated price path step by step.

Assume the price process follows **Geometric Brownian Motion (GBM)**. In discrete time with step size `dt`, one common simulation scheme is:

$$
S_{t+dt} = S_t \cdot \exp\left(\left(\mu - \frac{1}{2}\sigma^2\right)dt + \sigma \sqrt{dt}\, Z_t\right),
\quad Z_t \sim \mathcal{N}(0,1).
$$

Implement a generator function `gbm_prices(S0, mu, sigma, dt, n_steps)` that yields a stream of prices.

1. Use `numpy.random.normal(0, 1)` to generate the normal shocks \(Z_t\).
2. The generator should start at `S0` and yield prices one-by-one using `yield`.
3. Yield tuples `(t, S_t)` where `t` is the current time (starting at `0.0`) and `S_t` is the current simulated price.
4. Demonstrate that your generator is lazy by:
   - creating the generator object,
   - calling `next()` a few times manually,
   - and then iterating over the remaining values in a `for` loop.
5. Using the stream of generated prices, compute the **running mean** of the simulated price (online, without storing all prices in a list). Print the final running mean after the simulation ends.

**Do not store the full simulated path in memory.**


In [ ]:
# Your solution

### Exercise 2: Type Hinting 

Your task is to **add type hints** to the code in the next two cells (one basic, one slightly harder).  
Do not change the logic. Only add annotations and any necessary imports from `typing`.

Useful references (Official Python Documentation):
- [typing — Support for type hints](https://docs.python.org/3/library/typing.html)
- [Built-in generic types (`list[int]`, `dict[str, float]`, ...)](https://docs.python.org/3/library/stdtypes.html)

What you are expected to use:
- `Union` types via `A | B` (or `Union[A, B]`)
- `Any` for values where you cannot be precise
- Container types like `list[...]`, `dict[..., ...]`, `tuple[...]`
- `Callable[...]` for function arguments
- `None` types via `T | None`

**Exercise:** Add type hints (and required imports) to the two functions in the code cells.

In [ ]:
# Exercise 2.1

def format_value(x, decimals=2, missing="NA"):
    if x is None:
        return missing
    if isinstance(x, (int, float)):
        return f"{x:.{decimals}f}"
    return str(x)


def mean(values):
    if not values:
        raise ValueError("values must not be empty")
    return sum(values) / len(values)

In [ ]:
# Exercise 2.2

def parse_observation(row):
    if isinstance(row, dict):
        date = row["date"]
        value = row["value"]
    else:
        date, value = row

    return {"date": str(date), "value": float(value)}


def filter_and_transform(rows, predicate, transform=None):
    out = []
    for r in rows:
        obs = parse_observation(r)
        if predicate(obs):
            out.append(transform(obs) if transform is not None else obs)
    return out

### Exercise 3: Baseball Analytics (Sabermetrics)

You have just started an internship in a baseball analytics group. The team you support works in the "Moneyball spirit": using data-driven methods to evaluate players and make decisions under uncertainty and limited budgets.   
To get you onboarded, your supervisor asks you to build a small **object-oriented** analysis system that stores season totals and computes a few standard sabermetrics-style indicators.

Teams like the **Los Angeles Dodgers** are often highlighted as modern, analytics-driven organizations. Your job here is not to "predict the World Series", but to build clean tooling that makes analysis reproducible and easy to extend.

**Further reading (optional):**
- Sabermetrics overview and metric definitions: [FanGraphs Sabermetrics Library](https://library.fangraphs.com/)
- OPS definition (used for ranking): [FanGraphs: OPS](https://library.fangraphs.com/offense/ops/)
- Moneyball background: [Michael Lewis (official site)](https://www.michaellewiswrites.com/) and [Moneyball (book)](https://en.wikipedia.org/wiki/Moneyball:_The_Art_of_Winning_an_Unfair_Game)


#### Background: batting totals and simple metrics (needed for this exercise)

You will work with **season totals** (aggregated counts). The following abbreviations are used:

- `AB` (At-Bats): number of official batting attempts  
- `H` (Hits): number of times the player gets a hit  
- `2B` (Doubles): hits where the batter reaches second base  
- `3B` (Triples): hits where the batter reaches third base  
- `HR` (Home Runs): hits where the batter scores directly  
- `BB` (Walks): batter reaches first base due to four balls  
- `HBP` (Hit By Pitch): batter is awarded first base after being hit  
- `SF` (Sacrifice Flies): a plate appearance resulting in an out with a run scoring (used in OBP denominator)

From these totals, compute the following **performance metrics**:

- **Batting Average (BA)**  
  $$
  BA = \frac{H}{AB}
 $$

- **On-Base Percentage (OBP)**  
  $$
  OBP = \frac{H + BB + HBP}{AB + BB + HBP + SF}
 $$

- **Slugging Percentage (SLG)**  
  First compute singles:  
  $$
  1B = H - 2B - 3B - HR
  $$  
  Then total bases:  
  $$
  TB = 1B + 2\cdot 2B + 3\cdot 3B + 4\cdot HR
  $$  
  And:
  $$
  SLG = \frac{TB}{AB}
  $$

- **OPS (On-base Plus Slugging)**  
  $$
  OPS = OBP + SLG
 $$

If a denominator is zero, return `0.0` for that metric.

#### Tasks

##### 1. Create a `Person` class
- Attributes: `name`, `age`
- Method to display basic information
- Implement `__str__` for readable output

##### 2. Create a `Player` class (inherits from `Person`)
- Attributes: `player_id`, `position`
- Maintain a **private** stats container holding batting totals (counts such as `AB`, `H`, `BB`, `HR`, ...)
- Provide access to the stats through a `@property` (read-only; do not expose the private container directly)
- Implement a method that updates season totals by adding new game totals (counts)

##### 3. Implement player performance metrics (computed properties)
Implement computed properties for:
- Batting Average (BA)
- On-Base Percentage (OBP)
- Slugging Percentage (SLG)
- OPS = OBP + SLG

##### 4. Compare players by performance
- Implement comparison so players can be sorted by performance (use OPS as the default ranking)

##### 5. Create a `Team` class
- Attributes: `name` and a container of `players`
- Methods:
  - add players
  - list players
  - return top-n players by OPS
- Implement a team-level metric (e.g. average OPS across players)

##### 6. Create an `Analyst` class (inherits from `Person`)
- Attributes: `analyst_id`, `department`
- Method: produce a ranking report for a given team (top players + team summary)

##### 7. Test scenario
Create:
- one team
- one analyst
- multiple players with different season totals

Then:
- add players to the team
- update at least one player with additional stats
- print a readable report and show player sorting works


In [ ]:
# Your solution

### Exercise 4: Formula 1 Data Science

You have been offered a short internship in the data science unit of a Formula 1 team. Your group supports race-weekend decisions by combining telemetry, lap-time data, and simulation-based strategy evaluation. Your job is to build a small **object-oriented** system that stores lap times for multiple drivers and produces simple performance summaries.

This is inspired by how modern teams operate at scale: data engineering + modeling + fast reporting for engineers and strategists. A high-profile example is **Oracle Red Bull Racing**, where analytics and simulation play a central role in race preparation and strategy.

**Further reading (optional):**
- “Turning data into decisions” (Oracle Red Bull Racing): [Oracle blog](https://blogs.oracle.com/connect/oracle-red-bull-racing-turns-data-into-decisions)
- Hands-on (optional): [Oracle LiveLabs workshop](https://livelabs.oracle.com/ords/r/dbpm/livelabs/view-workshop?wid=909)

One core task in race analysis is turning raw lap time sequences into decision-ready metrics (pace and consistency). Your goal here is not realism, but clean, reusable code structure that matches how analysis tooling is organized.

#### Tasks

##### 1. Create a `Person` class
- Attributes: `name`, `age`
- Implement `__str__`

##### 2. Create a `Driver` class (inherits from `Person`)
- Attributes: `driver_id`, `team_name`
- Maintain a **private** container of lap times (seconds)
- Provide access through a `@property` (do not expose the private container directly)
- Method to add lap times

##### 3. Implement driver metrics (computed properties)
Implement computed properties for:
- Best lap time
- Average lap time
- A simple consistency metric (e.g. standard deviation of lap times)

##### 4. Compare drivers by performance
- Implement comparisons so drivers can be sorted by performance (lower average lap time is better)

##### 5. Create a `Session` class
- Attributes: `name` (e.g. `"FP1"`), `track`, and a container of drivers
- Methods:
  - register drivers
  - record laps for a driver
  - compute a leaderboard

##### 6. Create an `Engineer` class (inherits from `Person`)
- Attributes: `employee_id`, `area` (e.g. `"Performance"`)
- Method: produce a session report (leaderboard + summary stats)

##### 7. Test scenario
Create:
- one session with a track name
- one engineer
- multiple drivers with lap times

Then:
- record laps (for each driver)
- print a leaderboard and show sorting works


In [ ]:
# Your solution

### Optional Exercise: OOP Practice with `turtle` 

In the lecture, we introduced the core OOP ideas you will repeatedly use in applied work: **classes**, **objects**, **state**, and **methods**. In empirical projects, these ideas help you structure code so that it is readable, reusable, and easier to test.

For an optional, hands-on way to practice, Python’s built-in `turtle` library provides immediate visual feedback: when an object’s state changes (position, heading, pen state), you can see the result instantly.   
This makes it a useful sandbox for building intuition about how methods operate on object state.

**Official references:**
- [turtle — Turtle graphics (Python docs)](https://docs.python.org/3/library/turtle.html)
- [Turtle methods (`forward`, `left`, `penup`, `pendown`, ...)](https://docs.python.org/3/library/turtle.html#turtle-methods)

What to focus on (application of the OOP basics):
- Define a class that *encapsulates* a turtle object and exposes higher-level methods.
- Use instance attributes to represent **state** (e.g., current position, step counter).
- Separate responsibilities: one object draws, another controls logic (optional).

Possible, small extensions:
- Add a `steps` counter and a method that moves the turtle and updates the counter.
- Add a method `draw_square(size)` that draws a square starting at the turtle’s current position.
- Add a method `random_walk(n_steps, step_length)` that performs a random walk (left/right turns), while tracking turns.

In [ ]:
# Starting Point
import turtle
import random

class TurtleRobot:
    def __init__(self, step_length=20):
        self.t = turtle.Turtle()
        self.t.shape("turtle")
        self.t.speed(0)
        self.step_length = step_length
        self.steps = 0
        self.turns = 0

    def step(self):
        self.t.forward(self.step_length)
        self.steps += 1

    def turn_left(self):
        self.t.left(90)
        self.turns += 1

    def turn_right(self):
        self.t.right(90)
        self.turns += 1

    def random_walk(self, n_steps=50):
        for _ in range(n_steps):
            if random.random() < 0.5:
                self.turn_left()
            else:
                self.turn_right()
            self.step()

screen = turtle.Screen()
screen.title("TurtleRobot Demo")

robot = TurtleRobot(step_length=25)
robot.random_walk(n_steps=60)

print("Steps:", robot.steps)
print("Turns:", robot.turns)

screen.mainloop()